In [1]:
from youtubesearchpython import VideosSearch, CustomSearch, VideoDurationFilter
import youtubesearchpython
import librosa
import numpy as np
import pandas as pd
import pytube
import os
from pydub import AudioSegment
import speech_recognition
import pandas as pd
import re

In [2]:
recognizer = speech_recognition.Recognizer()
recognizer.energy_threshold = 300

In [3]:
def search_youtube(query, duration='long', limit=3):
    video_links = []
    if duration == 'long':
        vid_duration = VideoDurationFilter.long
    else:
        vid_duration = VideoDurationFilter.short
        
    results = CustomSearch(query, vid_duration, limit)
    for result in results.result()['result']:
        video_links.append(result['link'])
        
    return video_links

In [4]:
def download_yt_vids(link, save_path, file_name=''):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    youtube = pytube.YouTube(link)
    
    if file_name != '':
        youtube.title = file_name
    else:
        youtube.title = youtube.title.replace(' ', '_').lower()
    
    youtube.title = re.sub('[^a-zA-Z0-9 \n\.]', '_', youtube.title)
    
    video = youtube.streams.filter(only_audio=True).first()
    path = video.download(save_path)

    return path

In [5]:
def recognize_speech(audio_path):
    audio_file = speech_recognition.AudioFile(audio_path)
    try:
        with audio_file as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data=audio_data, language='ms')
    except:
        text = ''
        
    return text

In [6]:
def convert_to_mp3(aud_path):
    audio = AudioSegment.from_file(aud_path)
    new_aud_path = aud_path.replace('.wav', '.mp3')
    audio.export(new_aud_path, format='mp3')
    os.remove(aud_path)

In [7]:
def load_and_seg_audio(audio_path, seg_path, duration=10):
    audio = AudioSegment.from_file(audio_path)
    audio_length = len(audio)
    
    ms_dur = duration * 1000
    dur_list = list(range(0, len(audio), ms_dur))
    
    audio_filename = os.path.splitext(os.path.basename(audio_path))[0]
    wav_path = seg_path
    if not os.path.exists(wav_path):
        os.makedirs(wav_path)
    
    splitted_audio_paths = []
    for index, dur in enumerate(dur_list):
        if index == len(dur_list) - 1:
            break
        
        split_audio_path = wav_path + '/' + audio_filename + '_' + str(index) + '.wav'
        spliited_audio = audio[dur: dur_list[index + 1]]
        spliited_audio.export(split_audio_path, format='wav')
        splitted_audio_paths.append(split_audio_path)
    
    return splitted_audio_paths

In [8]:
video_links = search_youtube('baca berita', limit=3)
vid_save_path = 'data/vids/'

In [9]:
for index, vid_link in enumerate(video_links):
    print('Running video: {}'.format(vid_link))
    path = download_yt_vids(vid_link, vid_save_path)
    title = os.path.basename(path).split('.')[0]

    labels_path = 'data/' + title + '/labels/'
    audio_path = 'data/' + title + '/sound'
    spliited_audio_paths = load_and_seg_audio(path, audio_path)

    audio_filename = os.path.splitext(os.path.basename(spliited_audio_paths[0]))[0]
    txt_write_path = labels_path
    if not os.path.exists(txt_write_path):
        os.makedirs(txt_write_path)

    for index, aud_path in enumerate(spliited_audio_paths):
        text = recognize_speech(aud_path)
        if text != '':
            with open(txt_write_path + '/' + audio_filename + '_' +  str(index) + '.txt', 'w') as txt_file:
                txt_file.write(text)
                
            convert_to_mp3(aud_path) # convert back to mp3 from wav file
        else:
            os.remove(aud_path)

Running video: https://www.youtube.com/watch?v=Xwhd8aOFafs
Running video: https://www.youtube.com/watch?v=rCl1GsCsMu8
Running video: https://www.youtube.com/watch?v=OZ0CPT20j6Q
